In [1]:
import numpy as np
import pandas as pd
import re
import os

In [24]:
def getPaths(path):
    paths = []
    dir_list = []
    for root, dirs, files in os.walk(path):
        if len(dirs) < 1:
            for i in range(len(files)):
                path = '/'
                path_list = (root+'/'+files[i]).split('\\')
                path = path.join(path_list)
                paths.append(path)
        else:
            dir_list.append(dirs)
    if len(dir_list) < 1:
        return paths, dir_list
    return paths, dir_list[0]

In [41]:
def createImageIndex(path, force=False):
    result_file_path = "data_indices.csv"
    if os.path.isfile(result_file_path) and not force:
        return
    result_frame = []
    paths, dirs = getPaths(path)
    for patient in dirs:
        patient_filter = re.compile("(.*?)\/" + patient + "\/.*")
        patient_paths = list(filter(patient_filter.match, paths))
        volume_filter = re.compile("(\d+)(?!.*\d)")
        num_volumes = list(filter(volume_filter.match, patient_paths)) 
        print(num_volumes)
        # num_volumes = max(list(map(int,num_volumes)))
        print(num_volumes)
        for image in patient_paths:
            split_list = image.split("_")
            number = int(split_list[-1].split(".")[0])
            slice_num = number % 48
            volume_num = number // num_volumes
            result_frame.append([patient, volume_num, slice_num, image])
    df = pd.DataFrame(result_frame, columns= ["Patient", "Volume", "Slice", "ImagePath"])
    df.to_csv(result_file_path)

In [43]:
['D:/iCAT_IMAGES/001/001_20_1.dcm', 'D:/iCAT_IMAGES/001/001_20_10.dcm', 'D:/iCAT_IMAGES/001/001_20_100.dcm', 'D:/iCAT_IMAGES/001/001_20_1000.dcm', 'D:/iCAT_IMAGES/001/001_20_1001.dcm', 'D:/iCAT_IMAGES/001/001_20_1002.dcm', 'D:/iCAT_IMAGES/001/001_20_1003.dcm', 'D:/iCAT_IMAGES/001/001_20_1004.dcm', 'D:/iCAT_IMAGES/001/001_20_1005.dcm', 'D:/iCAT_IMAGES/001/001_20_1006.dcm', 'D:/iCAT_IMAGES/001/001_20_1007.dcm', 'D:/iCAT_IMAGES/001/001_20_1008.dcm', 'D:/iCAT_IMAGES/001/001_20_1009.dcm', 'D:/iCAT_IMAGES/001/001_20_101.dcm', 'D:/iCAT_IMAGES/001/001_20_1010.dcm', 'D:/iCAT_IMAGES/001/001_20_1011.dcm', 'D:/iCAT_IMAGES/001/001_20_1012.dcm', 'D:/iCAT_IMAGES/001/001_20_1013.dcm', 'D:/iCAT_IMAGES/001/001_20_1014.dcm', 'D:/iCAT_IMAGES/001/001_20_1015.dcm', 'D:/iCAT_IMAGES/001/001_20_1016.dcm', 'D:/iCAT_IMAGES/001/001_20_1017.dcm', 'D:/iCAT_IMAGES/001/001_20_1018.dcm', 'D:/iCAT_IMAGES/001/001_20_1019.dcm', 'D:/iCAT_IMAGES/001/001_20_102.dcm', 'D:/iCAT_IMAGES/001/001_20_1020.dcm', 'D:/iCAT_IMAGES/001/001_20_1021.dcm', 'D:/iCAT_IMAGES/001/001_20_1022.dcm', 'D:/iCAT_IMAGES/001/001_20_1023.dcm', 'D:/iCAT_IMAGES/001/001_20_1024.dcm', 'D:/iCAT_IMAGES/001/001_20_1025.dcm', 'D:/iCAT_IMAGES/001/001_20_1026.dcm', 'D:/iCAT_IMAGES/001/001_20_1027.dcm', 'D:/iCAT_IMAGES/001/001_20_1028.dcm', 'D:/iCAT_IMAGES/001/001_20_1029.dcm', 'D:/iCAT_IMAGES/001/001_20_103.dcm', 'D:/iCAT_IMAGES/001/001_20_1030.dcm', 'D:/iCAT_IMAGES/001/001_20_1031.dcm', 'D:/iCAT_IMAGES/001/001_20_1032.dcm', 'D:/iCAT_IMAGES/001/001_20_1033.dcm', 'D:/iCAT_IMAGES/001/001_20_1034.dcm', 'D:/iCAT_IMAGES/001/001_20_1035.dcm', 'D:/iCAT_IMAGES/001/001_20_1036.dcm', 'D:/iCAT_IMAGES/001/001_20_1037.dcm', 'D:/iCAT_IMAGES/001/001_20_1038.dcm', 'D:/iCAT_IMAGES/001/001_20_1039.dcm', 'D:/iCAT_IMAGES/001/001_20_104.dcm', 'D:/iCAT_IMAGES/001/001_20_1040.dcm', 'D:/iCAT_IMAGES/001/001_20_1041.dcm', 'D:/iCAT_IMAGES/001/001_20_1042.dcm', 'D:/iCAT_IMAGES/001/001_20_1043.dcm', 'D:/iCAT_IMAGES/001/001_20_1044.dcm', 'D:/iCAT_IMAGES/001/001_20_1045.dcm', 'D:/iCAT_IMAGES/001/001_20_1046.dcm', 'D:/iCAT_IMAGES/001/001_20_1047.dcm', 'D:/iCAT_IMAGES/001/001_20_1048.dcm', 'D:/iCAT_IMAGES/001/001_20_1049.dcm', 'D:/iCAT_IMAGES/001/001_20_105.dcm', 'D:/iCAT_IMAGES/001/001_20_1050.dcm', 'D:/iCAT_IMAGES/001/001_20_1051.dcm', 'D:/iCAT_IMAGES/001/001_20_1052.dcm', 'D:/iCAT_IMAGES/001/001_20_1053.dcm', 'D:/iCAT_IMAGES/001/001_20_1054.dcm', 'D:/iCAT_IMAGES/001/001_20_1055.dcm', 'D:/iCAT_IMAGES/001/001_20_1056.dcm', 'D:/iCAT_IMAGES/001/001_20_1057.dcm', 'D:/iCAT_IMAGES/001/001_20_1058.dcm', 'D:/iCAT_IMAGES/001/001_20_1059.dcm', 'D:/iCAT_IMAGES/001/001_20_106.dcm', 'D:/iCAT_IMAGES/001/001_20_1060.dcm', 'D:/iCAT_IMAGES/001/001_20_1061.dcm', 'D:/iCAT_IMAGES/001/001_20_1062.dcm', 'D:/iCAT_IMAGES/001/001_20_1063.dcm', 'D:/iCAT_IMAGES/001/001_20_1064.dcm', 'D:/iCAT_IMAGES/001/001_20_1065.dcm', 'D:/iCAT_IMAGES/001/001_20_1066.dcm', 'D:/iCAT_IMAGES/001/001_20_1067.dcm', 'D:/iCAT_IMAGES/001/001_20_1068.dcm', 'D:/iCAT_IMAGES/001/001_20_1069.dcm', 'D:/iCAT_IMAGES/001/001_20_107.dcm', 'D:/iCAT_IMAGES/001/001_20_1070.dcm', 'D:/iCAT_IMAGES/001/001_20_1071.dcm', 'D:/iCAT_IMAGES/001/001_20_1072.dcm', 'D:/iCAT_IMAGES/001/001_20_1073.dcm', 'D:/iCAT_IMAGES/001/001_20_1074.dcm', 'D:/iCAT_IMAGES/001/001_20_1075.dcm', 'D:/iCAT_IMAGES/001/001_20_1076.dcm', 'D:/iCAT_IMAGES/001/001_20_1077.dcm', 'D:/iCAT_IMAGES/001/001_20_1078.dcm', 'D:/iCAT_IMAGES/001/001_20_1079.dcm', 'D:/iCAT_IMAGES/001/001_20_108.dcm', 'D:/iCAT_IMAGES/001/001_20_1080.dcm', 'D:/iCAT_IMAGES/001/001_20_1081.dcm', 'D:/iCAT_IMAGES/001/001_20_1082.dcm', 'D:/iCAT_IMAGES/001/001_20_1083.dcm', 'D:/iCAT_IMAGES/001/001_20_1084.dcm', 'D:/iCAT_IMAGES/001/001_20_1085.dcm', 'D:/iCAT_IMAGES/001/001_20_1086.dcm', 'D:/iCAT_IMAGES/001/001_20_1087.dcm', 'D:/iCAT_IMAGES/001/001_20_1088.dcm', 'D:/iCAT_IMAGES/001/001_20_1089.dcm', 'D:/iCAT_IMAGES/001/001_20_109.dcm', 'D:/iCAT_IMAGES/001/001_20_1090.dcm', 'D:/iCAT_IMAGES/001/001_20_1091.dcm', 'D:/iCAT_IMAGES/001/001_20_1092.dcm', 'D:/iCAT_IMAGES/001/001_20_1093.dcm', 'D:/iCAT_IMAGES/001/001_20_1094.dcm', 'D:/iCAT_IMAGES/001/001_20_1095.dcm', 'D:/iCAT_IMAGES/001/001_20_1096.dcm', 'D:/iCAT_IMAGES/001/001_20_1097.dcm', 'D:/iCAT_IMAGES/001/001_20_1098.dcm', 'D:/iCAT_IMAGES/001/001_20_1099.dcm', 'D:/iCAT_IMAGES/001/001_20_11.dcm', 'D:/iCAT_IMAGES/001/001_20_110.dcm', 'D:/iCAT_IMAGES/001/001_20_1100.dcm', 'D:/iCAT_IMAGES/001/001_20_1101.dcm', 'D:/iCAT_IMAGES/001/001_20_1102.dcm', 'D:/iCAT_IMAGES/001/001_20_1103.dcm', 'D:/iCAT_IMAGES/001/001_20_1104.dcm', 'D:/iCAT_IMAGES/001/001_20_1105.dcm', 'D:/iCAT_IMAGES/001/001_20_1106.dcm', 'D:/iCAT_IMAGES/001/001_20_1107.dcm', 'D:/iCAT_IMAGES/001/001_20_1108.dcm', 'D:/iCAT_IMAGES/001/001_20_1109.dcm', 'D:/iCAT_IMAGES/001/001_20_111.dcm', 'D:/iCAT_IMAGES/001/001_20_1110.dcm', 'D:/iCAT_IMAGES/001/001_20_1111.dcm', 'D:/iCAT_IMAGES/001/001_20_1112.dcm', 'D:/iCAT_IMAGES/001/001_20_1113.dcm', 'D:/iCAT_IMAGES/001/001_20_1114.dcm', 'D:/iCAT_IMAGES/001/001_20_1115.dcm', 'D:/iCAT_IMAGES/001/001_20_1116.dcm', 'D:/iCAT_IMAGES/001/001_20_1117.dcm', 'D:/iCAT_IMAGES/001/001_20_1118.dcm', 'D:/iCAT_IMAGES/001/001_20_1119.dcm', 'D:/iCAT_IMAGES/001/001_20_112.dcm', 'D:/iCAT_IMAGES/001/001_20_1120.dcm', 'D:/iCAT_IMAGES/001/001_20_1121.dcm', 'D:/iCAT_IMAGES/001/001_20_1122.dcm', 'D:/iCAT_IMAGES/001/001_20_1123.dcm', 'D:/iCAT_IMAGES/001/001_20_1124.dcm', 'D:/iCAT_IMAGES/001/001_20_1125.dcm', 'D:/iCAT_IMAGES/001/001_20_1126.dcm', 'D:/iCAT_IMAGES/001/001_20_1127.dcm', 'D:/iCAT_IMAGES/001/001_20_1128.dcm', 'D:/iCAT_IMAGES/001/001_20_1129.dcm', 'D:/iCAT_IMAGES/001/001_20_113.dcm', 'D:/iCAT_IMAGES/001/001_20_1130.dcm', 'D:/iCAT_IMAGES/001/001_20_1131.dcm', 'D:/iCAT_IMAGES/001/001_20_1132.dcm', 'D:/iCAT_IMAGES/001/001_20_1133.dcm', 'D:/iCAT_IMAGES/001/001_20_1134.dcm', 'D:/iCAT_IMAGES/001/001_20_1135.dcm', 'D:/iCAT_IMAGES/001/001_20_1136.dcm', 'D:/iCAT_IMAGES/001/001_20_1137.dcm', 'D:/iCAT_IMAGES/001/001_20_1138.dcm', 'D:/iCAT_IMAGES/001/001_20_1139.dcm', 'D:/iCAT_IMAGES/001/001_20_114.dcm', 'D:/iCAT_IMAGES/001/001_20_1140.dcm', 'D:/iCAT_IMAGES/001/001_20_1141.dcm', 'D:/iCAT_IMAGES/001/001_20_1142.dcm', 'D:/iCAT_IMAGES/001/001_20_1143.dcm', 'D:/iCAT_IMAGES/001/001_20_1144.dcm', 'D:/iCAT_IMAGES/001/001_20_1145.dcm', 'D:/iCAT_IMAGES/001/001_20_1146.dcm', 'D:/iCAT_IMAGES/001/001_20_1147.dcm', 'D:/iCAT_IMAGES/001/001_20_1148.dcm', 'D:/iCAT_IMAGES/001/001_20_1149.dcm', 'D:/iCAT_IMAGES/001/001_20_115.dcm', 'D:/iCAT_IMAGES/001/001_20_1150.dcm', 'D:/iCAT_IMAGES/001/001_20_1151.dcm', 'D:/iCAT_IMAGES/001/001_20_1152.dcm', 'D:/iCAT_IMAGES/001/001_20_1153.dcm', 'D:/iCAT_IMAGES/001/001_20_1154.dcm', 'D:/iCAT_IMAGES/001/001_20_1155.dcm', 'D:/iCAT_IMAGES/001/001_20_1156.dcm', 'D:/iCAT_IMAGES/001/001_20_1157.dcm', 'D:/iCAT_IMAGES/001/001_20_1158.dcm', 'D:/iCAT_IMAGES/001/001_20_1159.dcm', 'D:/iCAT_IMAGES/001/001_20_116.dcm', 'D:/iCAT_IMAGES/001/001_20_1160.dcm', 'D:/iCAT_IMAGES/001/001_20_1161.dcm', 'D:/iCAT_IMAGES/001/001_20_1162.dcm', 'D:/iCAT_IMAGES/001/001_20_1163.dcm', 'D:/iCAT_IMAGES/001/001_20_1164.dcm', 'D:/iCAT_IMAGES/001/001_20_1165.dcm', 'D:/iCAT_IMAGES/001/001_20_1166.dcm', 'D:/iCAT_IMAGES/001/001_20_1167.dcm', 'D:/iCAT_IMAGES/001/001_20_1168.dcm', 'D:/iCAT_IMAGES/001/001_20_1169.dcm', 'D:/iCAT_IMAGES/001/001_20_117.dcm', 'D:/iCAT_IMAGES/001/001_20_1170.dcm', 'D:/iCAT_IMAGES/001/001_20_1171.dcm', 'D:/iCAT_IMAGES/001/001_20_1172.dcm', 'D:/iCAT_IMAGES/001/001_20_1173.dcm', 'D:/iCAT_IMAGES/001/001_20_1174.dcm', 'D:/iCAT_IMAGES/001/001_20_1175.dcm', 'D:/iCAT_IMAGES/001/001_20_1176.dcm', 'D:/iCAT_IMAGES/001/001_20_1177.dcm', 'D:/iCAT_IMAGES/001/001_20_1178.dcm', 'D:/iCAT_IMAGES/001/001_20_1179.dcm', 'D:/iCAT_IMAGES/001/001_20_118.dcm', 'D:/iCAT_IMAGES/001/001_20_1180.dcm', 'D:/iCAT_IMAGES/001/001_20_1181.dcm', 'D:/iCAT_IMAGES/001/001_20_1182.dcm', 'D:/iCAT_IMAGES/001/001_20_1183.dcm', 'D:/iCAT_IMAGES/001/001_20_1184.dcm', 'D:/iCAT_IMAGES/001/001_20_1185.dcm', 'D:/iCAT_IMAGES/001/001_20_1186.dcm', 'D:/iCAT_IMAGES/001/001_20_1187.dcm', 'D:/iCAT_IMAGES/001/001_20_1188.dcm', 'D:/iCAT_IMAGES/001/001_20_1189.dcm', 'D:/iCAT_IMAGES/001/001_20_119.dcm', 'D:/iCAT_IMAGES/001/001_20_1190.dcm', 'D:/iCAT_IMAGES/001/001_20_1191.dcm', 'D:/iCAT_IMAGES/001/001_20_1192.dcm']

['D:/iCAT_IMAGES/001/001_20_1.dcm',
 'D:/iCAT_IMAGES/001/001_20_10.dcm',
 'D:/iCAT_IMAGES/001/001_20_100.dcm',
 'D:/iCAT_IMAGES/001/001_20_1000.dcm',
 'D:/iCAT_IMAGES/001/001_20_1001.dcm',
 'D:/iCAT_IMAGES/001/001_20_1002.dcm',
 'D:/iCAT_IMAGES/001/001_20_1003.dcm',
 'D:/iCAT_IMAGES/001/001_20_1004.dcm',
 'D:/iCAT_IMAGES/001/001_20_1005.dcm',
 'D:/iCAT_IMAGES/001/001_20_1006.dcm',
 'D:/iCAT_IMAGES/001/001_20_1007.dcm',
 'D:/iCAT_IMAGES/001/001_20_1008.dcm',
 'D:/iCAT_IMAGES/001/001_20_1009.dcm',
 'D:/iCAT_IMAGES/001/001_20_101.dcm',
 'D:/iCAT_IMAGES/001/001_20_1010.dcm',
 'D:/iCAT_IMAGES/001/001_20_1011.dcm',
 'D:/iCAT_IMAGES/001/001_20_1012.dcm',
 'D:/iCAT_IMAGES/001/001_20_1013.dcm',
 'D:/iCAT_IMAGES/001/001_20_1014.dcm',
 'D:/iCAT_IMAGES/001/001_20_1015.dcm',
 'D:/iCAT_IMAGES/001/001_20_1016.dcm',
 'D:/iCAT_IMAGES/001/001_20_1017.dcm',
 'D:/iCAT_IMAGES/001/001_20_1018.dcm',
 'D:/iCAT_IMAGES/001/001_20_1019.dcm',
 'D:/iCAT_IMAGES/001/001_20_102.dcm',
 'D:/iCAT_IMAGES/001/001_20_1020.

In [42]:
createImageIndex("D:/iCAT_IMAGES", force=True)

['D:/iCAT_IMAGES/001/001_20_1.dcm', 'D:/iCAT_IMAGES/001/001_20_10.dcm', 'D:/iCAT_IMAGES/001/001_20_100.dcm', 'D:/iCAT_IMAGES/001/001_20_1000.dcm', 'D:/iCAT_IMAGES/001/001_20_1001.dcm', 'D:/iCAT_IMAGES/001/001_20_1002.dcm', 'D:/iCAT_IMAGES/001/001_20_1003.dcm', 'D:/iCAT_IMAGES/001/001_20_1004.dcm', 'D:/iCAT_IMAGES/001/001_20_1005.dcm', 'D:/iCAT_IMAGES/001/001_20_1006.dcm', 'D:/iCAT_IMAGES/001/001_20_1007.dcm', 'D:/iCAT_IMAGES/001/001_20_1008.dcm', 'D:/iCAT_IMAGES/001/001_20_1009.dcm', 'D:/iCAT_IMAGES/001/001_20_101.dcm', 'D:/iCAT_IMAGES/001/001_20_1010.dcm', 'D:/iCAT_IMAGES/001/001_20_1011.dcm', 'D:/iCAT_IMAGES/001/001_20_1012.dcm', 'D:/iCAT_IMAGES/001/001_20_1013.dcm', 'D:/iCAT_IMAGES/001/001_20_1014.dcm', 'D:/iCAT_IMAGES/001/001_20_1015.dcm', 'D:/iCAT_IMAGES/001/001_20_1016.dcm', 'D:/iCAT_IMAGES/001/001_20_1017.dcm', 'D:/iCAT_IMAGES/001/001_20_1018.dcm', 'D:/iCAT_IMAGES/001/001_20_1019.dcm', 'D:/iCAT_IMAGES/001/001_20_102.dcm', 'D:/iCAT_IMAGES/001/001_20_1020.dcm', 'D:/iCAT_IMAGES/001

TypeError: unsupported operand type(s) for //: 'int' and 'list'